In [1]:
#movie rec using collaborative data and matrix factorization
import csv
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
from scipy import optimize
from sklearn.decomposition import TruncatedSVD
from sklearn.cross_validation import train_test_split

#some code from http://alexabate.github.io/2016/11/05/movie-lens.html
#some code from https://nikhilwins.wordpress.com/2015/09/18/movie-recommendations-how-does-netflix-do-it-a-9-step-coding-intuitive-guide-into-collaborative-filtering/

user_item_ratings = pd.read_csv('ratings.csv', sep=',')
#user_item_ratings.head()
number_of_unique_users = len(user_item_ratings['userId'].unique())
number_of_unique_movies = len(user_item_ratings['movieId'].unique())
number_of_ratings = len(user_item_ratings)
#print user_item_ratings
print number_of_unique_users
print number_of_unique_movies

ratingsMatrix=pd.pivot_table(user_item_ratings, values='rating', fill_value=0, columns='userId', index='movieId')#
#print ratingsMatrix.head

ratingsMatrix2=ratingsMatrix.iloc[:5000,:]

print ratingsMatrix2.shape

did_rate = (ratingsMatrix != 0) * 1


#n_movies=number_of_unique_movies
n_movies=ratingsMatrix.shape[0]
n_users=number_of_unique_users


#normalize_ratings from https://nikhilwins.wordpress.com/2015/09/18/movie-recommendations-how-does-netflix-do-it-a-9-step-coding-intuitive-guide-into-collaborative-filtering/
def normalize_ratings(ratings, did_rate):
    num_movies = ratings.shape[0]
    ratings_mean = np.zeros(shape = (num_movies, 1))
    ratings_norm = np.zeros(shape = ratings.shape)
    
    for i in range(0, num_movies):
        # Get all the indexes where there is a 1
        idx = np.where(did_rate.iloc[i,:] ==1)[0]
        # Calculate mean rating of ith movie only from users that gave a rating
        ratings_mean[i] = np.mean(ratings.iloc[i, idx])
         #ratings_mean[i]
        ratings_norm[i, idx] = ratings.iloc[i, idx] - ratings_mean[i]
        #(EKC: make sure these indices aren't off by one)
        #print i
    return (ratings_norm, ratings_mean)

ratings_norm, ratings_mean = normalize_ratings(ratingsMatrix, did_rate)

did_rate=did_rate.as_matrix()

svd = TruncatedSVD(n_components=3, n_iter=10, random_state=42)
X_tr = svd.fit_transform(ratings_norm)
print X_tr.shape

X_o = svd.inverse_transform(X_tr)
#that would be the predicted ratings
difference = X_o*did_rate-ratings_norm
cost = np.sum((difference)**2)/2
print cost

671
9066
(5000, 671)
(9066, 3)
35267.1025259


In [2]:
movie_info = pd.read_csv('movies.csv', sep=',')


In [3]:

#need a list of all the genres--basically just go through the rows and add whichever ones are unique
all_genres=set()
for row in range(0,movie_info.shape[0]):
    genres=movie_info['genres'][row]
    genres_split=genres.split('|')
    all_genres.update(genres_split)
print all_genres
all_genres=list(all_genres)
print all_genres[0]
    


set(['Mystery', 'Drama', 'Western', 'Sci-Fi', 'Horror', 'Film-Noir', 'Crime', 'Romance', 'Fantasy', 'Musical', 'Animation', 'War', 'Adventure', 'Action', '(no genres listed)', 'Comedy', 'Documentary', 'Children', 'Thriller', 'IMAX'])
Mystery


In [4]:
#just for kicks let's see how many different tags there are, maybe can use those too
movie_tags = pd.read_csv('tags.csv', sep=',')


In [5]:


all_unique_tags=movie_tags['tag'].unique() 
print all_unique_tags[10:20] 
inds=np.where(movie_tags['tag']==all_unique_tags[12]) 
inds= np.array(inds) 
print inds[0] 
print movie_tags.iloc[inds[0],:] 
print len(inds[0]) 
tag_frequencies=[] 
for r in range(0,len(all_unique_tags)): 
    inds=np.where(movie_tags['tag']==all_unique_tags[r]) 
    freq=len(inds[0]) 
    tag_frequencies.append(freq)


['uganda' 'Ron Howard' 'music' 'weird' 'Steve Martin' 'action' 'anime'
 'kung fu' 'drama' 'indie']
[  12  776  802 1274]
      userId  movieId    tag   timestamp
12        68     2174  music  1249808064
776      531     1088  music  1243454488
802      531     6863  music  1243454317
1274     583   112552  music  1430526450
4


In [6]:
tags_in_descriptions=pd.pivot_table(movie_tags, values='timestamp', fill_value=0, columns='tag', index='movieId')#
tags_in_desctiptions=(tags_in_descriptions != 0) * 1

In [7]:
#print tags_in_descriptions.shape
#print tags_in_descriptions.iloc[:,0]
movies_in_tags=movie_tags['movieId'].unique()
movies_in_ratings=user_item_ratings['movieId'].unique()
mit=set(movies_in_tags)
mir=set(movies_in_ratings)

users_in_tags=movie_tags['userId'].unique()
users_in_ratings=user_item_ratings['userId'].unique()
uit=set(users_in_tags)
uir=set(users_in_ratings)

differentu=uit-uir
differentu2=uir-uit
bothu=uit&uir
#print different
#print different2
print len(bothu) #=630
print len(differentu)
print len(differentu2)

#a LOT of movies, and most users, don't ovelap... 


61
0
610


In [8]:
n_features=len(all_genres)
 
n_movies_genres=len(movie_info['movieId'].unique())
genres_in_descriptions=pd.DataFrame(np.zeros([n_movies_genres, n_features]))
print genres_in_descriptions.shape
#just want to combine the movies
print set(movie_info['movieId'].unique()) - set(user_item_ratings['movieId'].unique())
print set(user_item_ratings['movieId'].unique()) - set(movie_info['movieId'].unique())
#some movies have genres but haven't been rated. ignoring them for now.
# DELETE UNRATED MOVIES
#this is slow but ony has to happen once
n_movies=len(user_item_ratings['movieId'].unique())
genresMatrix=np.zeros([n_movies, n_features])
#print ratings_norm.shape
for ii in range(0,n_movies):
    thisMovie=ratingsMatrix.index.values[ii]
    Index=np.where(movie_info['movieId']==thisMovie)
    genres=movie_info['genres'][Index[0]]
    #genres_split=genres.split('|')
    gInd=0
    for g in all_genres:
        if g in str(genres):
            genresMatrix[ii, gInd]=1
        gInd=gInd+1
    

(9125, 20)
set([148482, 144172, 55946, 123663, 111249, 111251, 6830, 110871, 103449, 6683, 102469, 4763, 25927, 112550, 7335, 114459, 5289, 118700, 8236, 8494, 123695, 5169, 164979, 82313, 155064, 160954, 127144, 8766, 8767, 132800, 132547, 132549, 48711, 7243, 161582, 114254, 56015, 161336, 131796, 34517, 41585, 5984, 111505, 4712, 42217, 132458, 128235, 127212, 115139, 49917, 163056, 164977, 138610, 6515, 107636, 94969, 158783, 39421, 6229])
set([])


In [9]:
ii=0
thisMovie=ratingsMatrix.index.values[ii]
Index=np.where(movie_info['movieId']==thisMovie)
genres=movie_info['genres'][Index[0]]
print Index[0]
print movie_info['genres'][Index[0]]
print str(genres)
print 'Animation' in str(genres)

[0]
0    Adventure|Animation|Children|Comedy|Fantasy
Name: genres, dtype: object
0    Adventure|Animation|Children|Comedy|Fantasy
Name: genres, dtype: object
True


In [10]:
#convert ratings to probability of liking movie
# first z-score them (express each rating in terms of the standard deviation of that user's ratings)
stddevs=np.std(ratings_norm,axis=0)
ratings_zscored=ratings_norm/(stddevs)
# then get the probability (note that this assumes the ratings are distributed normally)
from scipy.stats import norm
probs=norm.cdf(ratings_zscored)


In [11]:
#bayesian model

ProbabilityOfLikingMovies=probs[1:,:].T # 
wordsInAllDescriptions=genresMatrix[1:,:]
#(leave one out for testing--later decide on the best cross-validation method)

#log likelihood of a user liking a movie is the sum of P(like) for all movies containing each word in the movie's description
# movie's description=genre (and tags? but user would not have tagged a new movie. Can include all tags for each movie?)
weightedP=ProbabilityOfLikingMovies.dot(wordsInAllDescriptions) #
                #(1-by-movies)         *  (movies-by-featurewords)
                #(users-by-movies)         *  (movies-by-featurewords)
            
wordsInMovieDescription=genresMatrix[0,:]
    
logPLikeMovie=weightedP.dot(wordsInMovieDescription)
#(users-by-features * features-by-1?)
#sum the probability of liking each movie that contains each word--this is a matrix multiplication, right?


In [12]:
 print logPLikeMovie.shape
    
plt.plot(logPLikeMovie) #(not sure why that doesn't work)
# logPLikeMovie should be similar to ratings (as well as they can be predicted by genre)--might be off by scale/threshold

(671,)


In [16]:
x=100
print x % 100 == 0

True


In [23]:
# do this for every movie
logPAllMovies=np.zeros(ratingsMatrix.T.shape)
for ii in range(0,n_movies):
    ProbabilityOfLikingMovies=np.delete(probs, ii, 0).T # 
    wordsInAllDescriptions=np.delete(genresMatrix, ii, 0)
    #(leave one out for testing--later decide on the best cross-validation method)

    #log likelihood of a user liking a movie is the sum of P(like) for all movies containing each word in the movie's description
    # movie's description=genre (and tags? but user would not have tagged a new movie. Can include all tags for each movie?)
    weightedP=ProbabilityOfLikingMovies.dot(wordsInAllDescriptions) #
                    #(1-by-movies)         *  (movies-by-featurewords)
                    #(users-by-movies)         *  (movies-by-featurewords)
            
    wordsInMovieDescription=genresMatrix[ii,:]
    
    logPLikeMovie=weightedP.dot(wordsInMovieDescription)
    #(users-by-features * features-by-1?)
    
    logPAllMovies[:,ii]=logPLikeMovie
    
    #sum the probability of liking each movie that contains each word--this is a matrix multiplication, right?
    if ii % 50 == 0:
        print ii

    




0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500
4550
4600
4650
4700
4750
4800
4850
4900
4950
5000
5050
5100
5150
5200
5250
5300
5350
5400
5450
5500
5550
5600
5650
5700
5750
5800
5850
5900
5950
6000
6050
6100
6150
6200
6250
6300
6350
6400
6450
6500
6550
6600
6650
6700
6750
6800
6850
6900
6950
7000
7050
7100
7150
7200
7250
7300
7350
7400
7450
7500
7550
7600
7650
7700
7750
7800
7850
7900
7950
8000
8050
8100
8150
8200
8250
8300
8350
8400
8450
8500
8550
8600
8650
8700
8750
8800
8850
8900
8950
9000
9050


In [24]:
print logPLikeMovie.shape

(671,)


In [25]:
    
#convert the log likelihoods to z scores and then to ratings--fill in the ratings matrix 
predictedRatingsNorm=logPAllMovies.T
PRNmeans=np.mean(predictedRatingsNorm, axis=0)
PRNstddevs=np.std(predictedRatingsNorm,axis=0)

predictedRatingsNorm=(predictedRatingsNorm-PRNmeans)/(PRNstddevs)
#scale by the standard rating deviation
predictedRatingsNorm=predictedRatingsNorm * stddevs #(make sure that's the right way to actually do this multiplication)



In [27]:
print predictedRatingsNorm

[[ 0.0362708   0.04919502  0.03236356 ...,  0.02221964  0.04224194
   0.04633192]
 [-0.06441377 -0.0864576  -0.05706423 ..., -0.03900042 -0.07440113
  -0.07860348]
 [ 0.00276306  0.00317367  0.0024114  ...,  0.00163404  0.00306177
   0.0031217 ]
 ..., 
 [-0.04013762 -0.05453305 -0.03572076 ..., -0.02452507 -0.04664094
  -0.04992248]
 [ 0.05996287  0.08142199  0.05333464 ...,  0.03654046  0.06956238
   0.07443439]
 [-0.11408373 -0.15403039 -0.10136709 ..., -0.06932304 -0.13229217
  -0.14159482]]


In [31]:
# fill in missing data in ratingsNorm with predictedRatingsNorm, then do the matrix factorization

denseMatrix=np.empty_like(predictedRatingsNorm) #this is probably not how this is done
denseMatrix[:]=predictedRatingsNorm
Inds=np.where(did_rate==1)
denseMatrix[Inds]=ratings_norm[Inds]#probably all this syntax is off

# then do the collab filter
svd = TruncatedSVD(n_components=3, n_iter=10, random_state=42)
finalPredictedRatings = svd.fit_transform(denseMatrix)

#how well does it do?
 